Import Necessary Libraries

In [31]:
import pandas as pd

Load the Data (Modify if needed)

In [9]:
# Define file path
file_path = r'C:\Users\Mohamed Salah\Desktop\Datasets\HR\PerformanceRating.csv'
# Load the dataset from the uploaded file
df = pd.read_csv(file_path, encoding="utf-8")

# Display basic information about the dataset
print("Dataset Overview:")
print(df.info())  # Shows column names, data types, and missing values
print(df.head())  # Shows the first few rows of the dataset

# Display general information about the dataset, including:
print(df.info())

# Generate a summary report of dataset statistics
def data_summary(df):
    print("Dataset Summary Report")
    print("-" * 40)
    print(f"Total Rows: {df.shape[0]}")
    print(f"Total Columns: {df.shape[1]}")
    print("\nMissing Values per Column:\n", df.isnull().sum())
    print("\nData Types:\n", df.dtypes)
    print("-" * 40)

Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6709 entries, 0 to 6708
Data columns (total 11 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   PerformanceID                    6709 non-null   object
 1   EmployeeID                       6709 non-null   object
 2   ReviewDate                       6709 non-null   object
 3   EnvironmentSatisfaction          6709 non-null   int64 
 4   JobSatisfaction                  6709 non-null   int64 
 5   RelationshipSatisfaction         6709 non-null   int64 
 6   TrainingOpportunitiesWithinYear  6709 non-null   int64 
 7   TrainingOpportunitiesTaken       6709 non-null   int64 
 8   WorkLifeBalance                  6709 non-null   int64 
 9   SelfRating                       6709 non-null   int64 
 10  ManagerRating                    6709 non-null   int64 
dtypes: int64(8), object(3)
memory usage: 576.7+ KB
None
  PerformanceID EmployeeI

Handle Missing Values & Standardize Data Types

In [33]:
# Check for missing values before handling
print("Missing values before handling:")
print(df.isnull().sum())

# Check for missing values
print("Checking for missing values")
missing_values = df.isnull().sum()

# If there are no missing values, print a confirmation message
if missing_values.sum() == 0:
    print("No missing values found in the dataset.")
else:
    print("Missing values found:")
    print(missing_values)

df['reviewdate'] = pd.to_datetime(df['reviewdate'], errors='coerce')

Missing values before handling:
performanceid                         0
employeeid                            0
reviewdate                            0
environmentsatisfaction               0
jobsatisfaction                       0
relationshipsatisfaction              0
trainingopportunitieswithinyear       0
trainingopportunitiestaken            0
worklifebalance                       0
selfrating                            0
managerrating                         0
performance_gap                       0
review_year                           0
rating_category                       0
worklife_balance_category          1510
training_flag                         0
satisfaction_score                    0
dtype: int64
Checking for missing values
Missing values found:
performanceid                         0
employeeid                            0
reviewdate                            0
environmentsatisfaction               0
jobsatisfaction                       0
relationshipsatisfaction 

Remove Duplicate Rows & Standardize Column Names

In [ ]:
# Remove duplicate rows
# Check for duplicate rows
print("\nChecking for duplicate rows...")
duplicate_rows = df.duplicated().sum()

if duplicate_rows == 0:
    print("No duplicate rows found.")
else:
    print(f"Number of duplicate rows found: {duplicate_rows}")
    df.drop_duplicates(inplace=True)
    print("Duplicate rows removed.")

# Remove duplicate rows if found
df.drop_duplicates(inplace=True)

# Standardize column names: lowercase, replace spaces with underscores
df.columns = df.columns.str.strip().str.lower().str.replace(" ", "_")

print("Duplicates removed & column names standardized.")


Checking for duplicate rows...
No duplicate rows found.
Duplicates removed & column names standardized.


Detecting and Removing Outliers Using the IQR Method

In [21]:
# Select numerical columns
numeric_cols = df.select_dtypes(include=['number'])

# Compute IQR
Q1 = numeric_cols.quantile(0.25)
Q3 = numeric_cols.quantile(0.75)
IQR = Q3 - Q1

# Define outlier range
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Remove outliers
df = df[~((numeric_cols < lower_bound) | (numeric_cols > upper_bound)).any(axis=1)]

print("Outliers Removed Successfully!")


Outliers Removed Successfully!


Feature Engineering & Data Transformation

In [36]:
# Extract the Year from 'reviewdate'
df['review_year'] = df['reviewdate'].dt.year

# Calculate Performance Gap (Difference Between Manager & Self Ratings)
df['performance_gap'] = df['managerrating'] - df['selfrating']

# Categorize Self Rating into "Low", "Medium", and "High"
df['rating_category'] = pd.cut(df['selfrating'], 
                               bins=[0, 2, 4, 5], 
                               labels=['Low', 'Medium', 'High'])

# Categorize Work-Life Balance Scores
df['worklife_balance_category'] = pd.cut(df['worklifebalance'], 
                                         bins=[0, 2, 3, 4], 
                                         labels=['Poor', 'Average', 'Good'])

# Create a Flag for Employees Who Had Training in the Year
df['training_flag'] = df['trainingopportunitieswithinyear'].apply(lambda x: 1 if x > 0 else 0)

# Create a "Satisfaction Score" (Average of Job & Environment Satisfaction)
df['satisfaction_score'] = df[['jobsatisfaction', 'environmentsatisfaction']].mean(axis=1)

# Convert Employee & Performance IDs to String (If Needed)
df['employeeid'] = df['employeeid'].astype(str)
df['performanceid'] = df['performanceid'].astype(str)

# Handle Missing Values (Optional)
df.fillna({'performance_gap': 0, 'review_year': df['review_year'].median()}, inplace=True)

# Display Final Data Preview
print(df.head())
print(df.info())  # Ensure transformations are correct

  performanceid employeeid reviewdate  environmentsatisfaction  \
0          PR01  79F7-78EC 2013-01-02                        5   
1          PR02  B61E-0F26 2013-01-03                        5   
2          PR03  F5E3-48BB 2013-01-03                        3   
3          PR04  0678-748A 2013-01-04                        5   
4          PR05  541F-3E19 2013-01-04                        5   

   jobsatisfaction  relationshipsatisfaction  trainingopportunitieswithinyear  \
0                4                         5                                1   
1                4                         4                                1   
2                4                         5                                3   
3                3                         2                                2   
4                2                         3                                1   

   trainingopportunitiestaken  worklifebalance  selfrating  managerrating  \
0                           0          

Saving the Cleaned Dataset

In [35]:
# Save the cleaned dataset to the same folder with a new name
cleaned_file_path = r'C:\Users\Mohamed Salah\Desktop\Datasets\HR\Cleaned_PerformanceRating.csv'
df.to_csv(cleaned_file_path, index=False)
df.to_csv('Cleaned_PerformanceRating.csv', index=False)